# Reddit Posts
This notebook allows to interaact with the reddit posts application using different configurations as well as to observe the result sets. The notebook does comparisons of new Reddit posts vs the Reddit posts from the previous execution.
- new posts since last execution
- posts that are no longer in top 75 since last execution
- posts whose scores changed since last execution

A markdown section has been added above each notebook cell explaining what it does.

## Constants
Constants are defined to track the path locations in the application.

In [49]:
BASE_PATH = '/Users/johnrojas/Development/vcs/github/johnwrf/reddit_posts'
SRC_PATH = '/Users/johnrojas/Development/vcs/github/johnwrf/reddit_posts/src'
DATA_PATH = '/Users/johnrojas/Development/vcs/github/johnwrf/reddit_posts/data'

NEW_POSTS_LISTING = "new"
TOP_POSTS_LISTING = "top"

NEW_POSTS_COUNT = 100
TOP75_POSTS_COUNT = 75

POST_COLUMNS = [
    "author_fullname",
    "title",
    "name",
    "score",
    "created",
    "view_count",
    "id",
    "author",
    "url",
    "created_utc"
    ]

POST_COLUMNS_X = {POST_COLUMNS[i]+"_x":POST_COLUMNS[i] for i in range(len(POST_COLUMNS))}
POST_COLUMNS_Y = {POST_COLUMNS[i]+"_y":POST_COLUMNS[i] for i in range(len(POST_COLUMNS))}


## Imports and configuration
This section imports dependant modules and makes sure the notebook has access to the Reddit Posts application source.
Finally the logging module is configured so that application log statements are visible when each notbook cell is executed.

In [50]:
import datatable as dt
import pandas as pd

# patch to import source code
import sys
sys.path.append(SRC_PATH)
print(sys.path)

from apis.reddit.reddit_posts import RedditPosts
from apis.io.post_io import load_posts, save_posts
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

['/Users/johnrojas/Development/vcs/github/johnwrf/reddit_posts/notebooks', '/usr/local/anaconda3/lib/python39.zip', '/usr/local/anaconda3/lib/python3.9', '/usr/local/anaconda3/lib/python3.9/lib-dynload', '', '/usr/local/anaconda3/lib/python3.9/site-packages', '/usr/local/anaconda3/lib/python3.9/site-packages/aeosa', '/Users/johnrojas/Development/vcs/github/johnwrf/reddit_posts/src', '/Users/johnrojas/Development/vcs/github/johnwrf/reddit_posts/src', '/Users/johnrojas/Development/vcs/github/johnwrf/reddit_posts/src', '/Users/johnrojas/Development/vcs/github/johnwrf/reddit_posts/src', '/Users/johnrojas/Development/vcs/github/johnwrf/reddit_posts/src']


## Load new posts

This section loads the latest new posts and the new posts from the last execution

Note that the same code is used to load both new posts and top posts

In [61]:
reddit_new = RedditPosts(listing=NEW_POSTS_LISTING, limit=NEW_POSTS_COUNT, timeframe="hour")
new_posts = reddit_new.load(columns=POST_COLUMNS)
logging.info(f"loaded latest new posts count {new_posts.nrows}")

last_new_posts = load_posts(listing=NEW_POSTS_LISTING, columns=POST_COLUMNS, base_path=DATA_PATH)
logging.info(f"loaded previous new posts count {last_new_posts.nrows}")

2022-09-23 11:37:29,591 [INFO] loaded latest new posts count 100
2022-09-23 11:37:29,594 [INFO] loaded previous new posts count 100


## Load Top 75 posts

This section loads the latest top 75 posts and the top 75 posts from the last execution

Note that the same code is used to load both new posts and top posts

In [62]:
reddit_top = RedditPosts(listing=TOP_POSTS_LISTING, limit=TOP75_POSTS_COUNT, timeframe="hour")
top_posts = reddit_top.load(columns=POST_COLUMNS)
logging.info(f"loaded latest top posts count {top_posts.nrows}")

last_top_posts = load_posts(listing=TOP_POSTS_LISTING, columns=POST_COLUMNS, base_path=DATA_PATH)
logging.info(f"loaded previous top posts count {last_top_posts.nrows}")

2022-09-23 11:37:32,647 [INFO] loaded latest top posts count 75
2022-09-23 11:37:32,649 [INFO] loaded previous top posts count 75


## Save posts
This section saves the lastest new and top75 posts to files on disk

In [60]:
save_posts(listing=NEW_POSTS_LISTING, posts=new_posts, base_path=DATA_PATH)
save_posts(listing=TOP_POSTS_LISTING, posts=top_posts, base_path=DATA_PATH)

True

## Determine new posts since last run
A pandas dataframe is used because the datatable df does not yet support left/right outer joins
The datatable df can easily be converted to a pandas df
The pandas merge function is then used with the indicator=True option, 
which generates a new "_merge" column, with values: left_only, right_only and both

Once the merge is performed, all the latest new posts are marked with "_merge"="left_only", meaning those are the posts that only appeared in the latest results.

Finally, column names are cleaned up and only the new posts data is returned

In [55]:
df_this_run = new_posts.to_pandas()
df_last_run = last_new_posts.to_pandas()
df_new_posts = pd.merge(df_this_run, df_last_run, on=['id'], how="outer", indicator=True)
display(df_new_posts.groupby(['_merge'])['_merge'].count())

df_new_since_last = df_new_posts[df_new_posts['_merge'] == 'left_only']
df_new_since_last = df_new_since_last.rename(POST_COLUMNS_X, axis=1)
df_new_since_last = df_new_since_last[POST_COLUMNS]
df_new_since_last.head

_merge
left_only     100
right_only    100
both            0
Name: _merge, dtype: int64

<bound method NDFrame.head of    author_fullname_x                                            title_x  \
0        t2_acculm9t  what should I do with my hair? should I cut th...   
1        t2_5l5ozyqq              A warning message exists for a reason   
2        t2_sr2a6562              I saw this cute girl in a coffee shop   
3         t2_vqzbu2g                     They're praying on my downfall   
4        t2_ed9lzz4d        Galtier a une idée pour démarabouter le PSG   
..               ...                                                ...   
95       t2_dfmms38c  [FOR HIRE] I do portrait and landscape illustr...   
96       t2_6fz93vjt                              +10 Flame Tribe Lyn ?   
97       t2_3dlemb4v  Book review -- BOLD VENTURES: Thirteen Tales o...   
98       t2_m1mag9xy  Rivian R1T Drag Racing a Tesla Model 3 Perform...   
99       t2_brd6e4v9                  Iams 3.5-Pound Dry Cat Food $2.93   

       name_x  score_x           created_x  view_count_x      id  \
0

## Determine posts no longer in top 75 since last run
A pandas dataframe is used because the datatable df does not yet support left/right outer joins
The datatable df can easily be converted to a pandas df
The pandas merge function is then used with the indicator=True option, 
which generates a new "_merge" column, with values: left_only, right_only and both

Once the merge is performed, the previous posts that were part of top 75 are marked with "_merge"="right_only", meaning those are the posts that no longer appeared in the top75 results.

Finally, column names are cleaned up and only the new posts data is returned

In [56]:
df_this_run = top_posts.to_pandas()
df_last_run = last_top_posts.to_pandas()
df_top_posts = pd.merge(df_this_run, df_last_run, on=['id'], how="outer", indicator=True)
display(df_top_posts.groupby(['_merge'])['_merge'].count())

df_no_longer_top75 = df_top_posts[df_top_posts['_merge'] == 'right_only']
df_no_longer_top75 = df_no_longer_top75.rename(POST_COLUMNS_Y, axis=1)
df_no_longer_top75 = df_no_longer_top75[POST_COLUMNS]
df_no_longer_top75.head

_merge
left_only     75
right_only    75
both           0
Name: _merge, dtype: int64

<bound method NDFrame.head of     author_fullname                                              title  \
75      t2_6oaefno3  KaiCenat reacts to Ice Posieden tweet about Mi...   
76      t2_gfd0vegb  Sliker's ex gf on living with the Austin texas...   
77       t2_6viux77  Adrianah tweets thread with 5 more ppl sharing...   
78      t2_pjosalwp                          2.3 Trillion Interception   
79      t2_4dhra4m3                              Ranger's Rank Shakeup   
..              ...                                                ...   
145     t2_6x507zyd                  Yeah it's already been 2 years...   
146     t2_73lxcajb                                        Every time…   
147     t2_2z06hyp3                 Starting a Nelf in Classic be like   
148     t2_bpy1i16m                                       No Caption..   
149     t2_a5j7p838                                      Cursed Pacman   

          name  score             created view_count      id  \
75   t3_xj02cl  3

## Determine posts whose score changed
To determine posts whose scores changed, we combine the new and top 75 posts, drop duplicates.
Then, we filter the posts that appeared during both program executions and where the score changed

In [64]:
df_scores = pd.concat([df_top_posts, df_new_posts])
df_scores.drop_duplicates(subset=['id'])

df_scores = df_scores[(df_scores['_merge'] == 'both') & (df_scores['score_x'] != df_scores['score_y'])]

df_scores = df_scores.rename(POST_COLUMNS_X, axis=1)
df_scores = df_scores[POST_COLUMNS+['score_y']]
df_scores['score_change'] = df_scores.score - df_scores.score_y
df_scores[['title','score','score_y','score_change']]

,title,score,score_y,score_change
